# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst so we can identify trends and predict market movements effectively.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning", "context": "We need to identify trends and predict market movements for AAPL"}

> Entering new CrewAgentExecutor chain...
I need to gather real-time data on AAPL to analyze trends and predict market movements accurately.

Action: Read website content
Action Input: {"website_url": "https://finance.yahoo.com/quote/AAPL"} 

Apple Inc. (AAPL) Stock Price,

Final Answer: 
Based on the real-time data from Yahoo Finance for AAPL, the stock price for Apple Inc. (AAPL) is currently at $216.24, with a 1-year return of 20.91% and a market cap of $3.29 trillion. The company designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. AAPL offers various services such as Apple Arcade, Apple Fitness+, Apple Music, Apple News+, Apple TV+, and Apple Pay. It serves consumers, small and mid-sized businesses, education, enterprise, and government markets. Additionally, AAPL distributes third-party applications through the App Store and sells products through retail and online stores, as well as through third-party carriers, wholesalers, retailers, and resellers.

> Finished chain.
 

Based on the real-time data from Yahoo Finance for AAPL, the stock price for Apple Inc. (AAPL) is currently at $216.24, with a 1-year return of 20.91% and a market cap of $3.29 trillion. The company designs, manufacture

Final Answer: 
Apple Inc. (AAPL) designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, Mac, iPad, and various other products. It also provides services like AppleCare support, cloud services, Apple Arcade, Apple Music, Apple News+, Apple TV+, and Apple Pay. Apple serves consumers, small businesses, education, enterprise, and government markets. The company distributes third-party applications through the App Store and sells its products through retail and online stores, as well as through third-party carriers, wholesalers, retailers, and resellers. Apple Inc. was founded in 1976 and is headquartered in Cupertino, California.

> Finished chain.
 

Apple Inc. (AAPL) designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, Mac, iPad, and various other products. It also provides services like AppleCare support, cloud se

Final Answer: 
Based on the real-time data from Yahoo Finance for AAPL, the stock price for Apple Inc. (AAPL) is currently at 216.24 USD (+1.37%). The company has a market cap of 3.288 trillion USD with a 1-year return of 20.27%. AAPL has had a YTD return of 12.31% and a 5-year return of 325.19%. The stock has a PE ratio of 32.91 and an EPS of 6.57. Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers various services such as Apple Arcade, Apple Fitness+, Apple Music, Apple News+, Apple TV+, Apple Card, and Apple Pay, among others. Apple Inc. serves consumers, small and mid-sized businesses, and education, enterprise, and government markets. The stock is currently performing well in the market, and based on this information, it could be a potential trading opportunity for users with a higher risk tolerance and day trading preferences.

> Finished chain.
 

Based on the real-time data from Yahoo 

I need to gather information on what specific criteria should be considered when developing execution plans for the approved trading strategies for AAPL.

Action: Read website content
Action Input: {"website_url": "https://www.investopedia.com/trading-strategies-4689748"} 

Investing in US Markets
 Investing
 Stocks
 Cryptocurrency
 Bonds
 ETFs
 Options and Derivatives
 Commodities
 Trading
 Automated Investing
 Brokers
 Fundamental Analysis
 Markets
 View All
 Simulator
 Login / Portfolio
 Trade
 Research
 My Games
 Leaderboard
 Banking
 Savings Accounts
 Certificates of Deposit (CDs)
 Money Market Accounts
 Checking Accounts
 View All
 Personal Finance
 Budgeting and Saving
 Personal Loans
 Insurance
 Mortgages
 Credit and Debt
 Student Loans
 Taxes
 Credit Cards
 Financial Literacy
 Retirement
 View All
 Economy
 Government and Policy
 Monetary Policy
 Fiscal Policy
 Economics
 View All
 News
 Markets
 Companies
 Earnings
 CD Rates
 Mortgage Rates
 Economy
 Government
 Crypto
 ETFs


Final Answer: 

Based on the insights provided by the Data Analyst, potential trading strategies for AAPL that align with the user's risk tolerance include:
1. Day trading AAPL stock based on short-term market trends and price fluctuations.
- Develop a detailed execution plan for day trading AAPL stock, focusing on short-term market trends and price fluctuations. Monitor market news related to AAPL, the performance of day trading, and the US Markets to capitalize on opportunities.
2. Investing in AAPL for long-term growth and capital appreciation, considering the company's strong market position and product offerings.
- Develop a detailed execution plan for investing in AAPL for long-term growth and capital appreciation. Monitor the financial performance of AAPL, external factors like government policies and economic conditions, and major US stock exchanges to enhance overall returns.
3. Utilizing options trading strategies to leverage AAPL's volatility for potential higher returns.
- 

Final Answer: Unfortunately, the website content for the proposed day trading strategy for AAPL could not be accessed. I will need to search the internet for alternative sources of information to evaluate the risks associated with this trading strategy.

> Finished chain.
 

Unfortunately, the website content for the proposed day trading strategy for AAPL could not be accessed. I will need to search the internet for alternative sources of information to evaluate the risks associated with this trading strategy.

Final Answer:
Risk Analysis Report for AAPL Trading Strategies

Introduction:
This risk analysis report aims to evaluate the risks associated with the proposed trading strategies and execution plans for AAPL. The analysis will cover potential risks and provide recommendations for mitigation strategies to enhance overall decision-making and risk management.

Day Trading Strategy:
- Potential Risks:
1. Market Volatility: Day trading AAPL stock based on short-term market trends and

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for AAPL Trading Strategies

Introduction:
This risk analysis report aims to evaluate the risks associated with the proposed trading strategies and execution plans for AAPL. The analysis will cover potential risks and provide recommendations for mitigation strategies to enhance overall decision-making and risk management.

Day Trading Strategy:
- Potential Risks:
1. Market Volatility: Day trading AAPL stock based on short-term market trends and price fluctuations exposes investors to high market volatility, leading to potential losses.
2. Liquidity Risk: Limited liquidity in the market may affect the execution of trades and result in unfavorable prices for AAPL stock.
3. Regulatory Risk: Changes in regulations related to day trading may impact the strategy's feasibility and compliance requirements.

- Mitigation Recommendations:
1. Implement Stop-Loss Orders: Set predefined stop-loss levels to limit potential losses and protect capital in case of adverse price movements.
2. Diversify Portfolio: Spread investments across different asset classes to mitigate concentration risk and reduce overall portfolio volatility.
3. Stay Informed: Monitor market news, economic indicators, and AAPL-related events to make informed decisions and adapt to changing market conditions.

Long-Term Growth Investment:
- Potential Risks:
1. Market Risk: Fluctuations in the stock market may affect the long-term growth potential of AAPL, leading to lower capital appreciation.
2. Company-Specific Risks: Changes in Apple's business performance, product offerings, or market position may impact the investment thesis.
3. Economic Conditions: Macroeconomic factors like interest rates, inflation, and GDP growth can influence the overall returns from long-term investments.

- Mitigation Recommendations:
1. Conduct Fundamental Analysis: Evaluate Apple's financial performance, competitive position, and growth prospects to make informed investment decisions.
2. Consider Diversification: Include AAPL as part of a diversified portfolio to reduce single-stock risk and enhance overall returns.
3. Monitor External Factors: Stay updated on government policies, economic conditions, and major stock exchanges to identify potential risks and opportunities.

Options Trading Strategy:
- Potential Risks:
1. Volatility Risk: Leveraging AAPL's volatility through options trading may result in higher gains but also increases the risk of significant losses.
2. Counterparty Risk: Dealing with options contracts exposes investors to counterparty default risk, where the other party fails to fulfill their obligations.
3. Market Timing Risk: Timing the market correctly to execute options trades at the right price levels can be challenging and may lead to suboptimal outcomes.

- Mitigation Recommendations:
1. Risk Management Tools: Use risk management tools like options spreads, stop-loss orders, and position sizing to control risk exposure and protect capital.
2. Educate Yourself: Gain a deep understanding of options trading strategies, implied volatility, and options pricing to make informed decisions.
3. Monitor Market Sentiment: Track market sentiment, options volume, and open interest to gauge investor behavior and potential market trends.

Diversified Portfolio Approach:
- Potential Risks:
1. Correlation Risk: High correlation between AAPL and other assets in the portfolio may increase overall risk exposure and limit diversification benefits.
2. Market Risk: External market factors like interest rates, geopolitical events, and sector-specific risks can impact the performance of the diversified portfolio.
3. Overlapping Positions: Holding similar assets or securities with overlapping characteristics may reduce the effectiveness of diversification and increase concentration risk.

- Mitigation Recommendations:
1. Asset Allocation: Allocate assets across different asset classes, sectors, and geographic regions to achieve diversification and reduce correlation risk.
2. Rebalance Regularly: Periodically review and rebalance the portfolio to maintain the desired asset allocation and risk-return profile.
3. Stress Testing: Conduct stress tests to assess the portfolio's performance under different market scenarios and identify potential vulnerabilities.

Monitoring Market News and Events:
- Potential Risks:
1. Information Overload: Excessive information flow from market news and events may lead to decision paralysis or suboptimal trading choices.
2. Confirmation Bias: Relying too heavily on market news without critical analysis can reinforce preconceived notions and biases in decision-making.
3. Market Noise: Sorting through irrelevant or misleading information in market news can cloud judgment and impact trading strategies negatively.

- Mitigation Recommendations:
1. Filter Information: Identify key sources of market news and events relevant to AAPL and filter out noise to focus on actionable insights.
2. Use Data Analytics: Employ data analytics tools to analyze market trends, sentiment, and news patterns to make data-driven decisions.
3. Peer Review: Seek feedback from colleagues, industry experts, or mentors to validate trading ideas and challenge assumptions for robust decision-making.

Conclusion:
In conclusion, the risk analysis report provides a comprehensive evaluation of the potential risks associated with the proposed trading strategies and execution plans for AAPL. By identifying specific risks, understanding their impact, and recommending mitigation strategies, investors can make informed decisions and manage risks effectively to enhance overall returns and portfolio performance. It is crucial to stay vigilant, adapt to changing market conditions, and continuously monitor risk factors to navigate the dynamic landscape of trading AAPL successfully.